In [1]:
import pandas as pd
import numpy as np
import re

In [95]:
regex_regra = re.compile('{(.*?)}->(\w+)')
regex_feat_value = re.compile('([^,=]+)=([^,=]+),?')

In [96]:
with open('regras_tfidf_top10qg_stem.txt') as f:    
    regras = [regex_regra.match(a).groups() for a in f]
    regras = [(regex_feat_value.findall(cond),target) for cond,target in regras]

In [97]:
regras[60:70]

[([('and/or', 'alto'), ('graphic model', 'alto')], '151'),
 ([('dechter', 'alto'), ('graphic', 'alto')], '151'),
 ([('dechter', 'alto'), ('tabl', 'alto')], '151'),
 ([('dechter', 'alto'), ('make', 'baixo')], '151'),
 ([('dechter', 'alto'), ('weight', 'alto')], '151'),
 ([('dechter', 'alto'), ('graphic model', 'alto')], '151'),
 ([('dechter', 'alto'), ('describ', 'medio')], '151'),
 ([('dechter', 'alto'), ('simpl', 'baixo')], '151'),
 ([('dechter', 'alto'), ('idea', 'alto')], '151'),
 ([('comput scienc', 'baixo'), ('dechter', 'alto')], '151')]

In [4]:
tfidf_matrix = pd.read_csv('doc_term_matrix_tfidf_discrete.csv')
tfidf_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Columns: 4264 entries, aaai to community
dtypes: int64(1), object(4263)
memory usage: 18.5+ MB


In [7]:
def evaluateRule(rule, example):
    return all(map(lambda x: example[x[0]]==x[1], rule[0]))

In [81]:
evaluateRule(regras[0],tfidf_matrix.loc[28,:])

True

In [163]:
def Qg(rule,df,target='community'):    
    examples = df.apply(lambda x: evaluateRule(rule,x), axis=1)
    tp = np.sum(df.loc[examples,target]==int(rule[1]))
    fp = np.sum(examples) - tp
#    print(tp,fp,np.argwhere(examples))
    return tp/(fp+1.0)

In [94]:
Qg(regras[0],tfidf_matrix)

11.0

In [164]:
import time
def permutation_test(rule,matrix,target='community',repetition=1000,seed=time.time()):
    np.random.seed(int(seed))    
    df = matrix.copy()
    def randomtest(i):        
        df.loc[:,target] = np.random.permutation(df[target])     
        return Qg(rule,df,target)
    return np.fromfunction(np.vectorize(randomtest),shape=(repetition,))

In [167]:
with open('random_test_tfidf.csv','w') as f:
    pvalue = []
    rep=1000
    for rule in regras:
        a = permutation_test(rule,tfidf_matrix,repetition=rep,seed=0)
        f.write(','.join(map(lambda x: '{0:.3g}'.format(x),a)))
        f.write('\n')        
        f.flush()
        qg = Qg(rule,tfidf_matrix)
        pvalue.append((np.sum(a>qg)+1)/(rep+1.0))
print(','.join(map(lambda x: '{0:5e}'.format(x),pvalue)))

In [129]:
s = []
for i in range(90,100):
    s.append(tfidf_matrix.apply(lambda x: evaluateRule(regras[i],x),axis=1))
[np.where(a) for a in s]

[(array([243, 279, 286, 319, 331, 369, 370, 371, 472, 486]),),
 (array([243, 286, 331, 369, 370, 371, 425, 472, 486]),),
 (array([243, 285, 286, 331, 369, 370, 371, 472, 486]),),
 (array([185, 243, 279, 286, 302, 319, 370, 371, 472]),),
 (array([185, 243, 279, 302, 319, 331, 369, 370, 371]),),
 (array([ 40, 279, 286, 302, 331, 369, 370, 371, 472]),),
 (array([243, 279, 286, 302, 319, 331, 369, 425, 486]),),
 (array([243, 279, 302, 319, 370, 425, 472, 486]),),
 (array([185, 302, 319, 369, 370, 371, 425, 472]),),
 (array([185, 279, 286, 369, 370, 371, 425, 486]),)]

In [130]:
np.sum(reduce(lambda x,y: x|y,s))

15

In [44]:
artigos = pd.read_csv('titulo_doc_comunidade.csv',sep=';')
artigosPR = pd.read_csv('titulo_doc_comunidade_pagerank.csv',sep=';')
artigosGrau = pd.read_csv('titulo_doc_comunidade_degree.csv',sep=';')

In [131]:
t = !ls txt/*.txt
t = [re.sub('txt2?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t]
t = np.array(t,dtype='object')[np.where(reduce(lambda x,y: x|y,s))[0]]

In [132]:
artigos[(artigos.id.isin(t))].title.values

array([ 'Solving Influence Diagrams using HUGIN, Shafer-Shenoy and Lazy Propagation',
       'Evaluating Influence Diagrams using LIMIDs',
       'Using ROBDDs for Inference in Bayesian Networks with Troubleshooting as an Example',
       'A computational scheme for Reasoning in Dynamic Probabilistic Networks',
       'Making Sensitivity Analysis Computationally Efficient',
       'Nested Junction Trees',
       'Reduction of Computational Complexity in Bayesian Networks through Removal of Weak Dependencies',
       'The Cost of Troubleshooting Cost Clusters with Inside Information',
       'Myopic Value of Information in Influence Diagrams',
       'Analysis in HUGIN of Data Conflict', 'Optimal Junction Trees',
       'From Influence Diagrams to Junction Trees',
       'Approximations in Bayesian Belief Universe for Knowledge Based Systems',
       'Approximate Learning in Complex Dynamic Bayesian Networks',
       'Belief Update in CLG Bayesian Networks With Lazy Propagation'], dtype

In [134]:
artigosPR[artigosPR.community=='grupo 256'].title.values

array([ 'Using ROBDDs for Inference in Bayesian Networks with Troubleshooting as an Example',
       'A computational scheme for Reasoning in Dynamic Probabilistic Networks',
       'Making Sensitivity Analysis Computationally Efficient',
       'Nested Junction Trees',
       'Reduction of Computational Complexity in Bayesian Networks through Removal of Weak Dependencies',
       'Sensitivity Analysis for Threshold Decision Making with Dynamic Networks',
       'Pivotal Pruning of Trade-offs in QPNs',
       'Enhancing QPNs for Trade-off Resolution',
       'Computing Probability Intervals Under Independency Constraints',
       'How to Elicit Many Probabilities',
       'The Computational Complexity of Sensitivity Analysis and Parameter Tuning',
       'Learning Bayesian Network Parameters with Prior Knowledge about Context-Specific Qualitative Influences',
       'Pre-processing for Triangulation of Probabilistic Networks',
       'Analysing Sensitivity Data from Probabilistic Netwo

In [136]:
artigosGrau[artigosGrau.community=='grupo 256'].title.values

array([ 'Sensitivity Analysis for Threshold Decision Making with Dynamic Networks',
       'Pivotal Pruning of Trade-offs in QPNs',
       'Enhancing QPNs for Trade-off Resolution',
       'Computing Probability Intervals Under Independency Constraints',
       'How to Elicit Many Probabilities',
       'The Computational Complexity of Sensitivity Analysis and Parameter Tuning',
       'Learning Bayesian Network Parameters with Prior Knowledge about Context-Specific Qualitative Influences',
       'Pre-processing for Triangulation of Probabilistic Networks',
       'Making Sensitivity Analysis Computationally Efficient',
       'Analysing Sensitivity Data from Probabilistic Networks',
       'aHUGIN: A System Creating Adaptive Causal Probabilistic Networks',
       'Inference in Multiply Sectioned Bayesian Networks with Extended Shafer-Shenoy and Lazy Propagation',
       'Myopic Value of Information in Influence Diagrams',
       'Using ROBDDs for Inference in Bayesian Networks with T

In [40]:
np.sum(artigos[(artigos.id.isin(t))].title.isin(artigosPR[artigosPR.community=='grupo 6'].title))

11

In [31]:
tfidf_matrix.community[np.where(reduce(lambda x,y: x|y,s))[0]]

22     6
27     6
28     6
36     6
74     6
76     6
127    6
134    6
163    6
194    6
201    6
202    6
225    6
233    6
261    6
272    6
276    6
311    6
343    6
382    6
405    6
408    6
433    6
434    6
448    6
457    6
471    6
545    6
563    6
Name: community, dtype: int64

In [16]:
artigos[]

array([ 28,  36, 134, 163, 225, 233, 261, 272, 457, 545, 563])

In [23]:
from functools import reduce

In [30]:
np.where(reduce(lambda x,y: x|y,s))[0]

array([ 22,  27,  28,  36,  74,  76, 127, 134, 163, 194, 201, 202, 225,
       233, 261, 272, 276, 311, 343, 382, 405, 408, 433, 434, 448, 457,
       471, 545, 563])